### Import

In [1]:
import copy
from typing import Optional, Any, Union, Callable

import torch
from torch import Tensor
import torch.nn as n
from torch.nn import functional as F

### Define 

In [2]:
#class Transformer(
#    d_model: int = 512,
#    nhead: int = 8,
#    num_encoder_layers: int = 6,
#    num_decoder_layers: int = 6,
#    dim_feedforward: int = 2048,
#    dropout: float = 0.1,
#    activation: str | ((Tensor) -> Tensor) = F.relu,
#    custom_encoder: Any | None = None,
#    custom_decoder: Any | None = None,
#    layer_norm_eps: float = 0.00001,
#    batch_first: bool = False,
#    norm_first: bool = False,
#    device: Any | None = None,
#    dtype: Any | None = None
#)
# Transformer_v1 =  n.Transformer(d_model= 63, nhead=7,dim_feedforward= 512 ) 

### In network traffic

In [3]:
import numpy as np
import pandas as pd

#### Load data

In [4]:
input = np.load('../SHAP/Network/test/task1/sources/task1_ver_kai_traffic.npy')
input.shape
print ("")

In [5]:
columns = [ 'srcip_mean', 'Dstip_mean','pro_mean','srcport_mean','dstport_mean','IPLen_mean','iat_mean',
                    'srcip_std', 'Dstip_std','pro_std','srcport_std','dstport_std','IPLen_std','iat_std',
                    'srcip_skew', 'Dstip_skew','pro_skew','srcport_skew','dstport_skew','IPLen_skew','iat_skew',
                    'srcip_kurt', 'Dstip_kurt','pro_kurt','srcport_kurt','dstport_kurt','IPLen_kurt','iat_kurt',
                    'srcip_entropy', 'Dstip_entropy','pro_entropy','srcport_entropy','dstport_entropy','IPLen_entropy','iat_entropy',
                    'srcip_distinct', 'Dstip_distinct','pro_distinct','srcport_distinct','dstport_distinct','IPLen_distinct','iat_distinct',
                    'srcip_q1', 'Dstip_q1','pro_q1','srcport_q1','dstport_q1','IPLen_q1','iat_q1',
                    'srcip_q2', 'Dstip_q2','pro_q2','srcport_q2','dstport_q2','IPLen_q2','iat_q2',
                    'srcip_q3', 'Dstip_q3','pro_q3','srcport_q3','dstport_q3','IPLen_q3','iat_q3']

#### Positional Encoding (PE)

In [6]:
## PE(pos, 2i) = sin(pos/10000^(2i/d_model))
## PE(pos, 2i+1) = cos(pos/10000^(2i/d_model))
def pos_encoding_2d(pos, i, d_model):
    return np.sin(pos / (10000 ** ((2 * i)/d_model))) if i%2==0 else np.cos(pos / (10000 ** ((2 * i)/d_model)))

In [7]:
pos_encoding = np.zeros((input.shape[0], input.shape[1]))
for pos in range(input.shape[0]):
    for i in range(input.shape[1]):
        pos_encoding[pos, i] = pos_encoding_2d(pos, i, input.shape[1])

In [8]:
input_df = pd.DataFrame(input, columns=columns)
pos_encoding_df = pd.DataFrame(pos_encoding, columns=[f'{columns[i]}_i{i}' for i in range(pos_encoding.shape[1])])

In [9]:
input_and_pe_df = pd.concat([input_df, pos_encoding_df], axis=1)
# sort columns by name group with i
input_and_pe_df = input_and_pe_df.reindex(sorted(input_and_pe_df.columns), axis=1)
input_and_pe_df

,Dstip_distinct,Dstip_distinct_i36,Dstip_entropy,Dstip_entropy_i29,Dstip_kurt,Dstip_kurt_i22,Dstip_mean,Dstip_mean_i1,Dstip_q1,Dstip_q1_i43,...,srcport_q1,srcport_q1_i45,srcport_q2,srcport_q2_i52,srcport_q3,srcport_q3_i59,srcport_skew,srcport_skew_i17,srcport_std,srcport_std_i10
0,0.3125,0.000000,0.450024,1.000000,0.000870,0.000000,0.706029,1.000000,0.752568,1.000000,...,0.006964,1.000000,0.006964,0.000000e+00,0.952492,1.0,0.259959,1.000000,0.936947,0.000000
1,0.5000,0.000027,0.308895,1.000000,0.000775,0.001608,0.644879,0.734086,0.675188,1.000000,...,0.006964,1.000000,0.107562,2.493592e-07,0.105345,1.0,0.631059,0.999976,0.485006,0.053697
2,1.0000,0.000054,0.434337,1.000000,0.000869,0.003216,0.641983,0.077766,0.675188,1.000000,...,0.006964,1.000000,0.015611,4.987184e-07,0.952523,1.0,0.325189,0.999904,0.859678,0.107239
3,0.4375,0.000080,0.325028,1.000000,0.000363,0.004825,0.607709,-0.619913,0.424583,1.000000,...,0.007310,1.000000,0.015611,7.480776e-07,0.105376,1.0,0.568921,0.999783,0.541381,0.160472
4,0.6875,0.000107,0.530759,1.000000,0.000493,0.006433,0.688484,-0.987905,0.549886,1.000000,...,0.011288,1.000000,0.015611,9.974368e-07,0.105360,1.0,0.580505,0.999615,0.557854,0.213241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,0.7500,0.015935,0.487449,0.992398,0.000394,0.816482,0.569996,-0.903811,0.214584,0.999998,...,0.006964,0.999999,0.006996,1.481194e-04,0.901998,1.0,0.253620,-0.557092,0.898632,0.475404
595,0.3750,0.015961,0.916667,0.992373,0.000363,0.817409,0.662446,-0.372885,0.560043,0.999998,...,0.025888,0.999999,0.477126,1.483687e-04,0.909211,1.0,0.210021,-0.551316,0.916341,0.521959
596,0.2500,0.015988,0.871049,0.992347,0.000249,0.818335,0.514163,0.356351,0.241711,0.999998,...,0.006964,0.999999,0.473212,1.486181e-04,0.984528,1.0,0.209206,-0.545514,1.000000,0.567008
597,0.3750,0.016015,0.759176,0.992321,0.000210,0.819258,0.484212,0.896070,0.214584,0.999998,...,0.025770,0.999999,0.691951,1.488674e-04,0.935374,1.0,0.173489,-0.539686,0.877402,0.610421


In [10]:
encoder_input = input + pos_encoding
encoder_input_df = pd.DataFrame(encoder_input, columns=columns)
encoder_input_df

,srcip_mean,Dstip_mean,pro_mean,srcport_mean,dstport_mean,IPLen_mean,iat_mean,srcip_std,Dstip_std,pro_std,...,dstport_q2,IPLen_q2,iat_q2,srcip_q3,Dstip_q3,pro_q3,srcport_q3,dstport_q3,IPLen_q3,iat_q3
0,0.950989,1.706029,0.000248,1.405278,0.817458,1.361927,0.040960,1.534498,0.602707,1.000180,...,1.568289,0.107383,1.002692,1.000000,1.844647,0.000266,1.952492,0.952463,1.175333,0.027173
1,1.706914,1.378966,0.528945,1.043783,0.456669,1.280505,0.182118,1.354727,0.493680,0.997494,...,1.015475,0.107383,1.000167,0.890985,1.752568,0.000094,1.105345,0.105342,1.142000,0.001965
2,1.684969,0.719749,0.897883,0.988400,1.167098,1.360740,0.347013,1.573070,0.724927,0.989874,...,1.093489,0.117823,1.000774,0.890985,1.752568,0.000266,1.952523,0.952494,1.235333,0.003624
3,0.850233,-0.012204,0.995028,0.451679,1.030445,0.942522,0.512638,1.411785,0.795100,0.976872,...,1.106658,0.128263,1.004567,0.890985,1.752568,0.000094,1.105376,0.105388,1.084000,0.014587
4,0.066630,-0.299421,0.791257,0.041812,1.089031,0.705446,0.659146,1.322852,0.912605,0.959000,...,1.015475,0.059658,1.006584,0.890985,1.875000,0.000094,1.105360,0.105357,1.057333,0.026325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,0.478839,-0.333814,-0.902160,-0.043761,1.528565,1.242918,0.802261,1.028433,1.385919,0.331632,...,1.093629,0.107465,1.001928,0.891031,1.752568,0.000291,1.901998,0.901985,1.170667,0.038488
595,-0.068897,0.289561,-0.993590,-0.282683,1.011451,1.076499,0.798501,0.589505,1.446637,0.398628,...,1.061542,0.108211,1.093470,0.891031,1.798801,0.000291,1.909211,0.307634,1.072500,0.206601
596,-0.136735,0.870514,-0.784366,-0.459661,1.104395,1.065767,0.686864,0.777982,1.502047,0.463344,...,1.469097,0.069434,1.160231,0.891031,1.752568,0.000119,1.984528,0.963453,1.044167,0.242341
597,0.865031,1.380282,-0.337691,-0.451815,0.547929,1.111132,0.480157,0.439748,1.473457,0.526036,...,1.044189,0.100008,1.044087,0.891031,1.752568,0.000119,1.935374,0.727885,1.051000,0.056316


#### Encoder

In [11]:
encoder_layer_v1 = n.TransformerEncoderLayer(d_model=63, nhead=7) # 63 = 7*9, 7 is the number of features, 9 is the number of pos_encoding
Transformer_encoder_v1 = n.TransformerEncoder(encoder_layer_v1, num_layers=6) # 6 layers, 6 encoder layers
#encoder_output = Transformer_encoder_v1(encoder_input)
encoder_output = Transformer_encoder_v1(torch.tensor(encoder_input).float()) 

In [17]:
encoder_input.shape

(599, 63)

In [12]:
torch.tensor(encoder_input).float().shape

torch.Size([599, 63])

torch.Size([599, 63])

In [13]:
encoder_out_df = pd.DataFrame(encoder_output.detach().numpy(), columns=columns)
encoder_out_df

,srcip_mean,Dstip_mean,pro_mean,srcport_mean,dstport_mean,IPLen_mean,iat_mean,srcip_std,Dstip_std,pro_std,...,dstport_q2,IPLen_q2,iat_q2,srcip_q3,Dstip_q3,pro_q3,srcport_q3,dstport_q3,IPLen_q3,iat_q3
0,-0.294307,0.058356,-0.810110,-0.037075,-1.660672,1.181340,-1.292565,0.371583,2.539087,-0.607696,...,1.307051,-0.152760,1.655057,0.626035,-0.341576,-0.153888,-0.118160,0.779942,-0.582403,-2.633715
1,0.249362,0.312145,-0.584427,0.072152,-2.601218,0.927227,-1.410657,0.364984,2.060027,-0.114367,...,0.935099,-0.427112,1.198601,0.032862,-0.537999,-0.853722,-0.273686,-0.064236,-1.151217,-2.546087
2,0.149955,-0.391710,-0.469606,-0.173840,-1.329481,1.162296,-1.079049,0.302046,2.431442,-0.059744,...,1.054804,-0.398189,1.379614,0.391366,-0.541039,-0.620562,-0.206697,0.134729,-0.961477,-2.283271
3,0.127595,-0.209650,-0.013867,-0.558111,-2.353511,1.378668,-1.084646,0.306850,2.713183,0.425122,...,0.953013,0.638415,1.110054,0.188924,-0.286476,-0.895694,-0.638153,-0.254062,-0.752259,-2.506828
4,-0.058556,-0.571038,-0.319977,-0.540094,-2.219813,1.041012,-0.963330,0.397591,2.579782,0.208993,...,1.061367,0.047427,1.316923,0.727996,-0.171268,-0.613576,-0.308667,0.148833,-0.637596,-2.131054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,-0.159733,-0.342369,-1.184044,-0.473470,-1.750771,0.633800,-1.007770,0.353438,2.406055,-0.398056,...,1.186282,-0.374822,1.241207,0.626620,-0.558526,0.143873,0.278176,0.520972,-0.459362,-2.601965
595,-0.244676,-0.169073,-0.943639,-0.821024,-1.137453,0.839117,-0.984716,0.131871,2.237227,-0.128847,...,1.125305,-0.067566,1.321291,0.527335,-0.530454,-0.133370,-0.132514,0.566948,-0.222451,-2.508606
596,-0.511340,0.197359,-0.543449,-0.831994,-1.709975,0.990668,-1.034562,0.236209,2.732281,-0.462171,...,0.914711,-0.215432,1.090708,0.523495,-0.552427,0.189817,0.180078,0.515214,-0.528467,-2.642105
597,0.067655,0.164162,-0.798516,-0.549649,-2.492682,0.965447,-1.187999,-0.112310,2.728828,-0.028958,...,0.968287,-0.294410,0.893304,0.422831,-0.520705,-0.255502,0.143905,0.398172,-0.563520,-2.385000
